# pyjoins

This notebook aims to compare a few join algorithms using simple in-memory index tables and disk I/O with python 2.7

For this purposes, we are using simple CSV-like files containing the data that will be used for our joins. For indexing, we are using the default python dictionary for our hash index and the OOBtree from the BTrees module for our btree index. Also, the data used our tests is generated by a generator script at the project [repository](https://github.com/pboueke/pyjoins).

We will be testing nested loop joins, merge-joins and hash-joins. We will be testing for files (ordered and unordered) indexed with both the hash index and the btree index.


### Loading our dependencies

In [5]:
from BTrees.OOBTree import OOBTree
import datetime as dt

### And generating the data

Using the generator.py script (at the repository), we will generate 4 files:

1. data_ordered_primary.dat
2. data_ordered_secondary.dat
3. data_unordered_primary.dat
4. data_unordered_secondary.dat

Parameters used:

In [11]:
size = 100000
field_delimiter = "|"
record_size =2048

This results in two datasets, called "primary" and "secondary". The primary contains two columns, one for its key and another with random characters. The secondary three columnes: its primary key, its foreign key (the primary key from the related record of the primary dataset) and a random charachter string. All registers contain 2048 characters and each line 2049, as defined above. Each dataset is generated in two versions, one ordered and another unordered, where the ordering is set by the ascending order of the primary key of the primary dataset. There is one register by line, a  "size" number of lines, and all relations between the two datasets are 1 to 1.

### Creating indexes

For simplicity, we shall create two index tables: one for each index type and for the unordered secondary dataset. Note that we are not considering the sizes, as, in this exercise, both datasets have the same size and we are always starting our scans at the primary one.

In [18]:
# instantiate our tables
# for hashtables, we are using the default python dictionary
hash_table = {}
# for the btree, we are using the BTrees package
btree = OOBTree() 

line_counter = 0
with open ("data_unordered_secondary.dat") as f:
    for line in f:
        # "primary_key|foreign_key|data"
        l = line.split(field_delimiter)
        hash_table[l[1]] = line_counter
        btree.update({l[1]:line_counter})
        line_counter += 1

# checking the size
print "Hash table size: " + str(len(hash_table))
print "Btree size: " + str(len(btree))

Hash table size: 10000
Btree size: 10000


### Starting with a simple nested loop

We don't need the indexes here. We will simply scan the ordered primary dataset file, line by line, and, for each line, we will scan the unordered secondary dataset until we find our match. As this is just a comparison exercise, we will just compute the join and track the time, not keeping the results in memory. 

In [17]:
start = dt.datetime.now()
matches = 0
right = open("data_unordered_secondary.dat")
with open("data_ordered_primary.dat") as left:
    for line1 in left:
        l1 = line1.split(field_delimiter)
        for line2 in right:
            l2 = line2.split(field_delimiter)
            if l1[0] == l2[1]:
                joined = l1 + l2
                matches += 1
                right.seek(0)
                break
right.close()
end = dt.datetime.now()                    

print "Total joined registers: " + str(matches)
print "Time taken: " + str((end-start).total_seconds()) + "s"

Total joined registers: 10000
Time taken: 180.545651s


### Merge-join

A sort-merge join approach should yield much faster results, but it requires both inputs to be ordered. As we are simulating an enviroment where we could not simply load all the data into the main memory, we must